In [1]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.pipeline import make_pipeline

## Fonctions

In [2]:
def read_file(file):
    # lis message whatsapp dnas une liste de str
    x = open(file,'r', encoding = 'utf-8') #Ouvre le fichier texte dans une variable x pas encore exploitable

    y = x.read() #chunck de str qu'on doit séparer ligne par ligne

    content = y.splitlines() #Splitline convertit le chunck d'str en liste de str
    return content

In [3]:
def clean(chat):
    # Mets les messages écrit sur plusieurs lignes sur la même ligne
    pos = []
    for i in range(len(chat)-1,0,-1):
        
        if len(chat[i]) <23 or chat[i][2] != '/' or chat[i][11] != 'à':
            chat[i-1] += chat[i]
            pos.append(i)
            
    for i in pos:
        del chat[i]

    #supp messages auto de wa
    chat = [truc for truc in chat if ': ' in truc]

    # création dataframe
    df = pd.DataFrame(chat)

    df['Année'] = df[0].apply(lambda x: x.split(' à ')[0])
    df['Heure'] = df[0].apply(lambda x: x.split(' à ',1)[1].split(' - ')[0])
    df['Nom'] = df[0].apply(lambda x: x.split(' - ')[1].split(': ')[0])
    df['Message'] = df[0].apply(lambda x: x.split(': ')[1])
    df.drop(0,axis = 1,inplace=True)
    
    #supprétion des gifs et médias
    df.drop(df[df['Message'].str.contains('<Média')].index, inplace=True)

    #suppr message doublons
    df = df.drop_duplicates(['Message'],keep='first')

    #suppr message contenant moins de 3 caractères
    ind = df[df['Message'].apply(lambda x: len(x)<3)].index
    df.drop(ind,inplace=True)
    
    return df

## Préparation et néttoyage des données

In [4]:
spam = pd.read_csv('spam.csv')
spam

,labels,text,text_hi,text_de,text_fr
0,ham,"Go until jurong point, crazy.. Available only ...","Dakag बिंदु तक जाओ, पागल. केवल Bag Non महान वि...","Gehen Sie bis jurong Punkt, verrückt.. Verfügb...","Allez jusqu'à Jurong point, fou.. Disponible s..."
1,ham,Ok lar... Joking wif u oni...,ओके लामर.... if if uue पर.,Ok Lar... joking wif u oni...,J'ai fait une blague sur le wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Fktatatat 21 मई को प्राप्त करने के लिए मुफ्त प...,Freier Eintritt in 2 a wkly comp zum Gewinn FA...,Entrée libre dans 2 a wkly comp pour gagner FA...
3,ham,U dun say so early hor... U c already then say...,Uden इतना जल्दी कहते हैं... तो पहले से ही यूसी...,U dun sagen so früh... U c schon dann sagen...,U dun dit si tôt hor... U c déjà dire alors...
4,ham,"Nah I don't think he goes to usf, he lives aro...","नहीं, मुझे नहीं लगता कि वह हमारे लिए चला जाता ...","Nein, ich glaube nicht, dass er zu unsf geht, ...","Non, je ne pense pas qu'il va à usf, il vit da..."
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,यह 2 सेकंड है जब हमने 2 संपर्क की कोशिश की है....,"Dies ist das zweite Mal, dass wir versucht hab...",C'est la 2ème fois que nous avons essayé 2 con...
5568,ham,Will ü b going to esplanade fr home?,क्या कलाई घर का पता लगाने के लिए जा रही होगी?,"Wird u b gehen, um esplanade fr home?",Est-ce que ü b ira à l'esplanade en maison?
5569,ham,"Pity, * was in mood for that. So...any other s...","तो फिर, दूसरे सुझाव क्या हैं?","Schade, * war in Stimmung dafür. Also... irgen...","Dommage, * était d'humeur pour ça. Donc... d'a..."
5570,ham,The guy did some bitching but I acted like i'd...,आदमी कुछ कुतियािंग किया लेकिन मैं मैं कुछ और ख...,"Der Typ hat ein bisschen rumgeschnüffelt, aber...",Le type a fait une saloperie mais j'ai agi com...


Nous avons un jeu de données issu de https://www.kaggle.com/datasets/rajnathpatel/multilingual-spam-data
La tradution française est parfois approximative.
Nous allons extraire les spams puis récuperer le texte français ainsi que le label que nous allons renommer en 'Message' et 'Type'. Cela nous servira plus tard

In [5]:
spam = spam[spam['labels'] == 'spam']
spam = spam[['text_fr','labels']]
spam.rename(columns={'text_fr':'Message','labels':'Type'},inplace=True)

In [6]:
spam

,Message,Type
2,Entrée libre dans 2 a wkly comp pour gagner FA...,spam
5,FreeMsg Hey il ya ma chérie il a été 3 semaine...,spam
8,WINNER!! En tant que client de réseau apprécié...,spam
9,Avez-vous eu votre mobile 11 mois ou plus? U R...,spam
11,SIX chances de gagner CASH! De 100 à 20 000 li...,spam
...,...,...
5537,Vous voulez explicitement SEX en 30 secondes? ...,spam
5540,3 MOBILE SELON 0870 CHATLINES COMPRISES DANS L...,spam
5547,Avait votre contrat mobile 11 Mnths? Dernier M...,spam
5566,"RAPPEL DE O2: Pour obtenir 2,50 livres de créd...",spam


In [7]:
df = read_file('conv1.txt')
df[0:3]

['21/01/2022 à 01:20 - Les messages et les appels sont chiffrés de bout en bout. Aucun tiers, pas même WhatsApp, ne peut les lire ou les écouter. Appuyez pour en savoir plus.',
 '21/01/2022 à 01:20 - Kyle Jr: <Médias omis>',
 '21/01/2022 à 01:20 - Kyle Jr: Ça que tu cherchais?']

Les données de conv1 sont issus d'une conversation wa. Par soucis de confidentialité je n'en affiche qu'une partie.
On va transformer ces données bruts (list) en un dataframe exploitable gràce à la fonction 'clean'. Ces données serviront à définir des données non spam.

In [8]:
df = clean(df)

In [9]:
df.loc[[1]]

,Année,Heure,Nom,Message
1,21/01/2022,01:20,Kyle Jr,Ça que tu cherchais?


In [10]:
#suppr colonnes Année Heure Nom
df.drop(['Année','Heure','Nom'], axis=1, inplace =True)

#ajout col type
df['Type'] = ['Ham']*len(df)

#concaténation de df et spam
df = pd.concat([df,spam])

#réinitialiser index
df.reset_index(inplace=True,drop=True)

In [11]:
df['Type'].value_counts()/len(df)

Ham     0.916349
spam    0.083651
Name: Type, dtype: float64

Il y a 8.3% de spam dans le dataset. Faute de mieux on s'en contentera.

## détection spam

Pour la détection nous utiliseront la classe TfidfVectorizer

In [12]:
encode = LabelEncoder()
y = encode.fit_transform(df['Type'])

x_train,x_test,y_train,y_test = train_test_split(df["Message"],y,test_size=0.2)

trans_vect=TfidfVectorizer()
x_train_trans=trans_vect.fit_transform(x_train)
x_test_trans=trans_vect.transform(x_test)

In [13]:
# on définit deux modèles
modele_bayes=MultinomialNB()
modele_svm=SVC()

In [14]:
modele_bayes.fit(x_train_trans, y_train)
modele_svm.fit(x_train_trans, y_train)

SVC()

In [15]:
print("Accuracy pour naive Bayes:",
accuracy_score(y_test, modele_bayes.predict(x_test_trans)))

print("Accuracy pour SVC :", accuracy_score(y_test, modele_svm.predict(x_test_trans)))

Accuracy pour naive Bayes: 0.9804031354983203
Accuracy pour SVC : 0.9893617021276596


On constate que le modèle SVC a une meilleur précision nous allons donc le choisir

In [16]:
# on construit un pipeline de traitement
pipe_text=make_pipeline(TfidfVectorizer(),SVC())

# on l’ajuste à toutes les données
# sachant qu’on a déjà validé le modèle
pipe_text.fit(df["Message"],y)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()), ('svc', SVC())])

In [17]:
def message_filter(message) :
    arr_mess=np.array([message])
    result=encode.inverse_transform(pipe_text.predict(arr_mess))[0]
    print("Le message reçu est un :", result)

In [18]:
message_filter("J'ai acheté du poulet pour le diner. Tu viens toujours")
message_filter("votre colis a été envoyé. veuillez le vérifier et le recevoir. https://truc.com/arnaque")

Le message reçu est un : Ham
Le message reçu est un : spam
